<a href="https://colab.research.google.com/github/bhargabdhungel/woc_machine_learning/blob/main/NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# importing from drive
raw_training_data = pd.read_csv('/content/drive/MyDrive/woc/logistic_reg_data/emnist-letters-train.csv')
raw_testing_data = pd.read_csv('/content/drive/MyDrive/woc/logistic_reg_data/emnist-letters-test.csv')
training_data=raw_training_data.to_numpy()
testing_data=raw_testing_data.to_numpy()

In [ ]:
m=1000
testing_sample=500

In [ ]:
input_layer=784
hidden_layer=300
output_layer=26

In [ ]:
def g(x):
  if x.all():
    return 1/(1+np.exp(-x))
  else:
    return 1- 1/(1+np.exp(x)) 

In [ ]:
def dvt(t):
  return t*(1-t)

In [ ]:
# traing data
x=training_data[:m,1:]
x=(x-np.mean(x,axis=0))/255
y_=training_data[:m,0]-1
y=np.zeros((1,26))
for i in y_:
  t=np.zeros((26,))
  t[i.astype(int)]=1
  y=np.vstack((y,t))
y=y[1:,:]

In [ ]:
# testing data 
test_random = random.sample(range(0, testing_sample),testing_sample)
test_x=testing_data[:,1:]
test_x=(test_x-np.mean(test_x,axis=0))/255
test_y=testing_data[:,0]

In [ ]:
a0=x

In [ ]:
w1 = np.random.random((input_layer,hidden_layer))
b1=np.ones((m,hidden_layer))
z1=(a0@w1)
a1=z1+b1

In [ ]:
z1=a0@w1
b1=np.ones((m,1))
z1=np.hstack((b1,z1))
a1=g(z1)

In [ ]:
z2=a1@w2
b2=np.ones((m,1))
z2=np.hstack((np.ones((m,1)),z2))
a2=g(z2)

In [ ]:
iter=100

In [ ]:
alph=0.002

In [ ]:
d_a2.shape

In [ ]:
d_w2=(d_a2@w2.T)/m

In [ ]:
d_b2=np.sum(d_a2,axis=1)/m

In [ ]:
t=a1@w2
t=np.hstack((np.ones((m,1)),z2))
d_a1=(w2.T@d_a2)*(t*(1-t))

In [ ]:
d_w1=(d_a1@w1.T)/m

In [ ]:
d_b1=np.sum(d_a1,axis=1)/m

In [ ]:
# still trying could not complete this on time 